# Imports

In [171]:
import geojson as gj
import matplotlib.pyplot as plt
import folium as fl
import geopandas as gpd
import json
import pandas as pd
import networkx as nx
import sys
import haversine
import pickle

# Importing data and splitting columns

## Nodes

In [172]:
# Load the file "/Users/johanmasvie/programming/tiø4900/master_thesis/Scigrid_data/IGGIN_Nodes.geojson" into a GeoDataFrame
nodes_gdf = gpd.read_file('/Users/johanmasvie/programming/tiø4900/master_thesis/Scigrid_data/IGGIN_Nodes.geojson')

nodes_gdf.head()

,name,id,country_code,tags,param,method,geometry
0,N_0,NO_N_0,NO,{},"{'eic_code': None, 'elevation_m': None, 'exact...","{'eic_code': None, 'elevation_m': None, 'exact...",POINT (6.79720 65.10640)
1,N_1,NO_N_1,NO,{},"{'eic_code': None, 'elevation_m': None, 'exact...","{'eic_code': None, 'elevation_m': None, 'exact...",POINT (5.19357 59.33286)
2,N_2,NO_N_2,NO,{},"{'eic_code': None, 'elevation_m': None, 'exact...","{'eic_code': None, 'elevation_m': None, 'exact...",POINT (2.89575 60.56023)
3,N_3,NO_N_3,NO,{},"{'eic_code': None, 'elevation_m': None, 'exact...","{'eic_code': None, 'elevation_m': None, 'exact...",POINT (2.89614 60.78219)
4,N_4,NO_N_4,NO,{},"{'eic_code': None, 'elevation_m': None, 'exact...","{'eic_code': None, 'elevation_m': None, 'exact...",POINT (3.29837 54.78877)


## Pipelines

In [173]:
# Open the file: "/Users/johanmasvie/programming/tiø4900/master_thesis/Scigrid_data/IGGIN_PipeSegments.csv" as a GeoDataFrame: gdf
pipelines_gdf = gpd.read_file('/Users/johanmasvie/programming/tiø4900/master_thesis/Scigrid_data/IGGIN_PipeSegments.csv')

# Split the 'param' column into separate columns
pipelines_gdf['param'] = pipelines_gdf['param'].apply(lambda x: json.loads(x.replace("'", '"')))
param_df = pd.json_normalize(pipelines_gdf['param'])
pipelines_gdf = pipelines_gdf.drop(columns=['param']).join(param_df)

# Split the 'uncertainty' column into separate columns and add them to the GeoDataFrame, prefaced by the string 'uncertainty_'
pipelines_gdf['uncertainty'] = pipelines_gdf['uncertainty'].apply(lambda x: json.loads(x.replace("'", '"')))
uncertainty_df = pd.json_normalize(pipelines_gdf['uncertainty'])
uncertainty_df.columns = ['uncertainty_' + col for col in uncertainty_df.columns]
pipelines_gdf = pipelines_gdf.drop(columns=['uncertainty']).join(uncertainty_df)

# Split the 'method' column into separate columns and add them to the GeoDataFrame, prefaced by the string 'method_'
pipelines_gdf['method'] = pipelines_gdf['method'].apply(lambda x: json.loads(x.replace("'", '"')))
method_df = pd.json_normalize(pipelines_gdf['method'])
method_df.columns = ['method_' + col for col in method_df.columns]
pipelines_gdf = pipelines_gdf.drop(columns=['method']).join(method_df)

# Split the "country_code" column into two separate columns. "['ES', 'ES']" is an example of a value in the "country_code" column
pipelines_gdf['country_code'] = pipelines_gdf['country_code'].apply(lambda x: x.replace('[', '').replace(']', '').replace("'", '').split(', '))
country_code_df = pd.DataFrame(pipelines_gdf['country_code'].to_list(), columns=['country_code_1', 'country_code_2'])
pipelines_gdf = pipelines_gdf.drop(columns=['country_code']).join(country_code_df)

# Split the node_id column into two separate columns. "['INET_N_23', 'INET_N_295']" is an example of a value in the "node_id" column
pipelines_gdf['node_id'] = pipelines_gdf['node_id'].apply(lambda x: x.replace('[', '').replace(']', '').replace("'", '').split(', '))
node_id_df = pd.DataFrame(pipelines_gdf['node_id'].to_list(), columns=['node_id_1', 'node_id_2'])
pipelines_gdf = pipelines_gdf.drop(columns=['node_id']).join(node_id_df)

# Print the first few rows of the GeoDataFrame
pipelines_gdf.head()

,id,name,source_id,lat,long,comment,tags,geometry,diameter_mm,end_year,...,method_long_mean,method_max_cap_M_m3_per_d,method_max_pressure_bar,method_num_compressor,method_start_year,method_waterDepth_m,country_code_1,country_code_2,node_id_1,node_id_2
0,INET_PL_1,AlcazarDeSanJuan_Chinchilla,['INET_PL_1'],"[39.3900748, 38.892435]","[-3.3500984, -1.719831]",None,{},None,923.792529,2050,...,make_Attrib(latlong_mean),Lasso,Median,Median,make_Attrib(const),Lasso,ES,ES,INET_N_23,INET_N_295
1,INET_PL_3,Tarancon_Madrid,['INET_PL_3'],"[40.01, 40.223]","[-3.207, -3.681]",None,{},None,923.792529,2050,...,make_Attrib(latlong_mean),Lasso,Median,Median,make_Attrib(const),Lasso,ES,ES,INET_N_1460,INET_N_904
2,INET_PL_5,Alessandria_Cortemaggiore,['INET_PL_5'],"[44.991076, 44.90008]","[9.920891, 8.60944]",None,{},None,914.400000,2050,...,make_Attrib(latlong_mean),Lasso,raw,Median,make_Attrib(const),Lasso,IT,IT,INET_N_317,INET_N_26
3,INET_PL_7,Almendralejo_Badajoz,['INET_PL_7'],"[38.711202, 38.87665]","[-6.382539, -6.9714]",None,{},None,923.792529,2050,...,make_Attrib(latlong_mean),Lasso,Median,Median,make_Attrib(const),Lasso,ES,ES,INET_N_36,INET_N_108
4,INET_PL_9,Almendralejo_Zamora,['INET_PL_9'],"[38.711202, 41.506]","[-6.382539, -5.644]",None,{},None,923.792529,2050,...,make_Attrib(latlong_mean),Lasso,Median,Median,make_Attrib(const),Lasso,ES,ES,INET_N_36,INET_N_1653


## Fixing disconnected components

In [174]:
# Create a graph of the pipeline network
G = nx.Graph()

# Add the nodes to the graph
for i, row in nodes_gdf.iterrows():
    G.add_node(row['id'], pos=(row['geometry'].x, row['geometry'].y))

# Add the edges to the graph
for i, row in pipelines_gdf.iterrows():
    G.add_edge(row['node_id_1'], row['node_id_2'])

# Add a column to the nodes_gdf GeoDataFrame that contains a number symbolizing the connected component the node is a part of
nodes_gdf['connected_component'] = 0
for i, component in enumerate(nx.connected_components(G)):
    for node in component:
        nodes_gdf.loc[nodes_gdf['id'] == node, 'connected_component'] = i

# Save the graph object to a file
with open('pipeline_network.pkl', 'wb') as f:
    pickle.dump(G, f)

In [175]:
# Plot the graph using folium
m = fl.Map(location=[nodes_gdf['geometry'].y.mean(), nodes_gdf['geometry'].x.mean()], zoom_start=5)
for i, row in nodes_gdf.iterrows():
    fl.Marker([row['geometry'].y, row['geometry'].x], popup=row['id']).add_to(m)
for i, row in pipelines_gdf.iterrows():
    fl.PolyLine([(nodes_gdf.loc[nodes_gdf['id'] == row['node_id_1'], 'geometry'].values[0].y, nodes_gdf.loc[nodes_gdf['id'] == row['node_id_1'], 'geometry'].values[0].x),
                 (nodes_gdf.loc[nodes_gdf['id'] == row['node_id_2'], 'geometry'].values[0].y, nodes_gdf.loc[nodes_gdf['id'] == row['node_id_2'], 'geometry'].values[0].x)]).add_to(m)